In [1]:
import os

try:
  dbutils.fs.mount("s3a://bd-databricks-less-limited-shard-kjbhdfkljbnkjbdfkjbalkdnvubhbv/sequenceddata.csv")
except:
  pass

In [2]:
import pandas as pd
import pprint
import math
pp = pprint.PrettyPrinter(indent = 4)

#read from csv (TODO change this file to updated version)
df = pd.read_csv('./sequence_dec.csv')

columnlist = df.columns.tolist()

df

,Category,Questions,Field,Tasks,Assignment,Visibility,Timing,Minimum Time,Budget,"*Once payment schedule is added, task is created under ""Payment"" category"
0,General,Event Date,text,Discuss wedding weekend events with client,Planner,Yes,7 months,5,NaN,NaN
1,NaN,NaN,NaN,Register for gifts,Client,Yes,7 months,5,NaN,NaN
2,NaN,NaN,NaN,- Obtain your marriage license,Client,Yes,3 months,1,NaN,NaN
3,NaN,NaN,NaN,"Purchase gifts for wedding party, parents and ...",Client,Yes,2 months,1,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,Location,text,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,Event Start Time,text,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,Event End Time,text,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,Guest Count,text,Compile Guest list,Client,Yes,9 months,6,NaN,NaN
9,NaN,NaN,NaN,- Collect Guest email and physical addresses,Client,Yes,9 months,7,NaN,NaN


In [3]:
xy = df


#fill in Category, questions, field according to row above
xy['Category'].fillna(method='ffill', inplace=True)
xy['Questions'].fillna(method='ffill', inplace=True)   
xy['Field'].fillna(method='ffill', inplace=True) 

#delete rows with no tasks
xy = xy[xy.Tasks.notnull()]

#reset index to contingent numbers after deleting rows with no tasks
xy.reset_index(drop=True, inplace=True)

In [4]:

#record sequence of tasks
# group by (Category, Questions Field). If any one value changes, start new sequence from 1
index = [] #stores all sequence
i = 1
curr_category = xy['Category'][0]
curr_question = xy['Questions'][0]
curr_field = xy['Field'][0]

for j in range(len(xy)):
    c = xy['Category'][j]
    q = xy['Questions'][j]
    f = xy['Field'][j]
    
    if c != curr_category:
        i = 1
        curr_category = c
    if q != curr_question:
        i = 1
        curr_question = q
    if f != curr_field:
        i = 1
        curr_field = f
   
    index.append(i)
    i += 1
    
xy['Sequence'] = index #add sequence as a new column to visulize
xy

/Users/emily/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Category,Questions,Field,Tasks,Assignment,Visibility,Timing,Minimum Time,Budget,"*Once payment schedule is added, task is created under ""Payment"" category",Sequence
0,General,Event Date,text,Discuss wedding weekend events with client,Planner,Yes,7 months,5,NaN,NaN,1
1,General,Event Date,text,Register for gifts,Client,Yes,7 months,5,NaN,NaN,2
2,General,Event Date,text,- Obtain your marriage license,Client,Yes,3 months,1,NaN,NaN,3
3,General,Event Date,text,"Purchase gifts for wedding party, parents and ...",Client,Yes,2 months,1,NaN,NaN,4
4,General,Guest Count,text,Compile Guest list,Client,Yes,9 months,6,NaN,NaN,1
5,General,Guest Count,text,- Collect Guest email and physical addresses,Client,Yes,9 months,7,NaN,NaN,2
6,General,Guest Count,text,- RSVP deadline,Planner,Yes,1 Month,1,NaN,NaN,3
7,General,Guest Count,text,- Send Final guest count to Planner,Client,Yes,1 month,2 weeks,NaN,NaN,4
8,General,Guest Count,text,- Update floor plan based on final guest count...,Planner,No,4 weeks,4 weeks,NaN,NaN,5
9,General,Guest Count,text,Final table/seat assignments due,Client,Yes,1 month,3 weeks,NaN,NaN,6


In [5]:
#assign columns to lists
category = [x for x in xy['Category']]
question = [x for x in xy['Questions']]
field = [x for x in xy['Field']]
description = xy['Tasks']
assignment = [x for x in xy['Assignment']]
visibility = [x for x in xy['Visibility']]
timing = [x for x in xy['Timing']]
mintime = [x for x in xy['Minimum Time']]
sequence = [x for x in xy['Sequence']]

In [6]:
#get rid of "-" in front of description
description = [x.replace("- ", "", 1) for x in description]
#get rid of " " after description
description = [x.strip() for x in description]


timing_new = []
mintime_new = []
#timing and mintime 
for i in range(len(timing)):
    
    #if Null value, keep as null
    if isinstance(timing[i], float) or isinstance(mintime[i], float):
        timing_new.append(None)
        mintime_new.append(None)
        continue
    
    
    #detect number for timing and mintime
    timing_num = int(''.join(filter(str.isdigit, timing[i])))
    mintime_num = int(''.join(filter(str.isdigit, mintime[i])))
    
    #detect starting alphabet (unit) for timing and mintime
    unit_timing= ''.join(filter(lambda x: x.isalpha(), timing[i]))[0]
    mintime_alpha = ''.join(filter(lambda x: x.isalpha(), mintime[i]))

    #if no units specified for mintime, default MONTH
    if mintime_alpha == '' or mintime_alpha[0] == 'm':
        unit_mintime = 'm'
    else:
        unit_mintime = mintime_alpha[0]
    
    #change all units to DAYS
    if unit_timing == 'm':
        timing_new.append(timing_num * 30)
    elif unit_timing == 'w':
        timing_new.append(timing_num * 7)
    else:
        timing_new.append(timing_num)
        
    if unit_mintime == 'm':
        mintime_new.append(mintime_num * 30)
    elif unit_mintime == 'w':
        mintime_new.append(mintime_num * 7)
    else:
        mintime_new.append(mintime_num)
       
    #negate for bubble computation
    timing_new[i] *= -1
    mintime_new[i] *= -1

            
print(timing_new)
print(mintime_new)

#for bubble computation
timingratio = [-x/300 for x in timing_new if x is not None]
        
#visibility
# for i, v in enumerate(visibility):
#     if v != 'yes':
#         visibility[i] = 'no'

[-210, -210, -90, -60, -270, -270, -1, -30, -28, -30, -21, -21, -14, -300, -300, -270, -270, -270, -270, -120, -300, -210, -240, -210, -180, -150, -120, -120, -240, -2250, -240, -150, -150, -120, -180, -120, -60, -30, -210, -60, -90, -300, -300, None, -300, -300, -300, None, -300, -300, -300, -300, -300, -270, -90, -90, -300, -300, -300, -270, -90, -90, -90, -60, -1, -210, -120, -120, -120, -60, -210, -180, -270, -270, -2550, -150, -90, -90, -30, -21, -90, -90, -120, -270, -90, -120, -90, -180, -210, -120, -90, -90, None, -150, -150, -270, -270, -240, -90, -90, -30, -42, -30, -42, -30, -42, -300, -270, -270, -30, -270, -30, -240, -240, -180, -180, -240, -240, -240, None, -210, -210, -240, -240, -210, -240, -2250, -210, -180, -90, -90, -30, -28, -4, -210, -210, -180, -180, -150, -120, -120, -120, -90, -120, -120, -90, -63, -210, -180, -150, -180, -180, -180, -180, -150, -180, -1650, -150, -180, -180, -150, -240, -270, -270, -90, -270, -210, -1950, -3, -210, -210, -1950, -3, -150, -150, 

In [7]:
xy["newtime"] = timing_new
xy["newmin"] = mintime_new

newtime = [x for x in xy['newtime']]
newmin = [x for x in xy['newmin']]

newsequence = [i+1 for i in range(len(xy))]

xy["newsequence"] = newsequence

xy

/Users/emily/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/emily/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/emily/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum

,Category,Questions,Field,Tasks,Assignment,Visibility,Timing,Minimum Time,Budget,"*Once payment schedule is added, task is created under ""Payment"" category",Sequence,newtime,newmin,newsequence
0,General,Event Date,text,Discuss wedding weekend events with client,Planner,Yes,7 months,5,NaN,NaN,1,-210.0,-150.0,1
1,General,Event Date,text,Register for gifts,Client,Yes,7 months,5,NaN,NaN,2,-210.0,-150.0,2
2,General,Event Date,text,- Obtain your marriage license,Client,Yes,3 months,1,NaN,NaN,3,-90.0,-30.0,3
3,General,Event Date,text,"Purchase gifts for wedding party, parents and ...",Client,Yes,2 months,1,NaN,NaN,4,-60.0,-30.0,4
4,General,Guest Count,text,Compile Guest list,Client,Yes,9 months,6,NaN,NaN,1,-270.0,-180.0,5
5,General,Guest Count,text,- Collect Guest email and physical addresses,Client,Yes,9 months,7,NaN,NaN,2,-270.0,-210.0,6
6,General,Guest Count,text,- RSVP deadline,Planner,Yes,1 Month,1,NaN,NaN,3,-1.0,-30.0,7
7,General,Guest Count,text,- Send Final guest count to Planner,Client,Yes,1 month,2 weeks,NaN,NaN,4,-30.0,-14.0,8
8,General,Guest Count,text,- Update floor plan based on final guest count...,Planner,No,4 weeks,4 weeks,NaN,NaN,5,-28.0,-28.0,9
9,General,Guest Count,text,Final table/seat assignments due,Client,Yes,1 month,3 weeks,NaN,NaN,6,-30.0,-21.0,10


In [8]:

# Calling DataFrame constructor after zipping all lists

#can also do xy["newmin"] = mintime_new for example to make change in-place 
#instead of creating new df
df = pd.DataFrame(list(zip(description, question, field, timing, timingratio, mintime, assignment, visibility, category,sequence,newsequence,newtime,newmin)), 
               columns =['description', 'question', 'field', 'timing', 'timingratio', 'mintime', 'assignment', 'visibility', 'category', 'sequence','newsequence','newtime','newmin']) 
df



,description,question,field,timing,timingratio,mintime,assignment,visibility,category,sequence,newsequence,newtime,newmin
0,Discuss wedding weekend events with client,Event Date,text,7 months,0.700000,5,Planner,Yes,General,1,1,-210.0,-150.0
1,Register for gifts,Event Date,text,7 months,0.700000,5,Client,Yes,General,2,2,-210.0,-150.0
2,Obtain your marriage license,Event Date,text,3 months,0.300000,1,Client,Yes,General,3,3,-90.0,-30.0
3,"Purchase gifts for wedding party, parents and ...",Event Date,text,2 months,0.200000,1,Client,Yes,General,4,4,-60.0,-30.0
4,Compile Guest list,Guest Count,text,9 months,0.900000,6,Client,Yes,General,1,5,-270.0,-180.0
5,Collect Guest email and physical addresses,Guest Count,text,9 months,0.900000,7,Client,Yes,General,2,6,-270.0,-210.0
6,RSVP deadline,Guest Count,text,1 Month,0.003333,1,Planner,Yes,General,3,7,-1.0,-30.0
7,Send Final guest count to Planner,Guest Count,text,1 month,0.100000,2 weeks,Client,Yes,General,4,8,-30.0,-14.0
8,Update floor plan based on final guest count a...,Guest Count,text,4 weeks,0.093333,4 weeks,Planner,No,General,5,9,-28.0,-28.0
9,Final table/seat assignments due,Guest Count,text,1 month,0.100000,3 weeks,Client,Yes,General,6,10,-30.0,-21.0


In [9]:
#writing results to results.csv
content = df.to_csv()
with open('dec.csv','w') as file:
    print("writing")
    file.write(content)
    file.close()

writing
